## Preparations

### import modules

In [2]:
import math
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

### Prepare data

#### Prepare train_set

In [3]:
def ToWord(title):
    target_words=[]
    for word in title.split(' '):
        if word != '' and word != ' ':
            target_words.append(word)
        
    return target_words

In [4]:
def PrepareTrainSet():
    with open('Book3.csv','r') as r:
        reader=csv.reader(r)
        index=0
        train_set={}

            
        for row in reader:
            person=[] 
            title_sequence=[]
            #delete the empty element
            for element in row:
                if len(element) == 0:
                    break
                else:
                    person.append(element) #person: id+job_title_sequence
            title_sequence=person[1:]      #title_sequnce: job sequence without id
            max_length=len(title_sequence)-1
            for i in range(1,max_length+1):
                for j in range(0,len(title_sequence)-i):
                    data={}
                    data['input']=[]
                    for k in range(j,i+j):
                        data['input'].append(title_sequence[k])
#                        data['input'].append(str(EOS_token))
                    title=title_sequence[i+j]
                    data['target']=ToWord(title)
#                    data['target'].append(str(EOS_token))
                    train_set[index]=data
                    index += 1
           
    return train_set

In [5]:
train_set = PrepareTrainSet()

In [6]:
lenTrainSet=len(train_set)

In [7]:
lenTrainSet

315174

#### Wordbank & Titlebank

In [8]:
class Word():
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addTrainSet(self, train_set):
#        for word in sentence.split(' '):
#            self.addWord(word)
        for i in range(len(train_set)):
            for word in train_set[i]['target']:
                self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [9]:
class Title():
    def __init__(self):
        self.title2index = {}
        self.title2count = {}
        self.index2title = {0: "SOS", 1: "EOS"}
        self.n_titles = 2  # Count SOS and EOS

    def addTrainSet(self, train_set):
#        for word in sentence.split(' '):
#            self.addWord(word)
        for i in range(len(train_set)):
            for title in train_set[i]['input']:
                self.addTitle(title)

    def addTitle(self, title):
        if title not in self.title2index:
            self.title2index[title] = self.n_titles
            self.title2count[title] = 1
            self.index2title[self.n_titles] = title
            self.n_titles += 1
        else:
            self.title2count[title] += 1

In [10]:
wordbank=Word()
wordbank.addTrainSet(train_set)

In [11]:
titlebank=Title()
titlebank.addTrainSet(train_set)

#### Prepare Dataset

In [12]:
SOS_token = 0
EOS_token = 1

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
class PositionDataset(Dataset):
    
    def __init__(self,train_set):
        self.train_set=train_set
        
    def __len__(self):
        return len(self.train_set)
    
    def __getitem__(self,index):
        input_indexes=[]
        target_indexes=[]
        for item in self.train_set[index]['input']:
            input_indexes.append(titlebank.title2index[item])
        input_indexes.append(EOS_token)
        input_tensor = torch.tensor(input_indexes, dtype = torch.long, device = device).view(-1,1)
        for word in self.train_set[index]['target']:
            target_indexes.append(wordbank.word2index[word])
        target_indexes.append(EOS_token)
        target_tensor = torch.zeros(wordbank.n_words, dtype = torch.long, device = device)
        for index in target_indexes:
            target_tensor[index] = 1
        
        sample={'input':input_tensor, 'target':target_tensor}
        # input_tensors:[tensor1(title),tensor2(title)....]
        '''target_tensor: [
        [],
        []]'''
        return sample

## Model

### Positional Encoding

d_model = ninp (embedding dimension) <br>
nhid: the dimension of the feedforward network model in nn.TransformerEncoder <br>
nhead: the number of heads in the multiheadattention models  <br>
nlayers: the number of nn.TransformerEncoderLayer in nn.TransformerEncoder  <br>

In [15]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=100):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

#### questions: 
1. why the decoder is ntoken? so the input and output tokens are the same?
2. is ntokens here means the number of the words?

In [81]:
class TransformerModel(nn.Module):

    def __init__(self, n_titles, n_words, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout) #get the positional encoder
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout) #get a layer
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers) # get the encoder
        self.encoder = nn.Embedding(n_titles, ninp) #embedding layer
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, n_words)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output

In [17]:
n_titles = titlebank.n_titles

In [18]:
n_titles

48951

In [19]:
n_words = wordbank.n_words

In [20]:
n_words

20187

## Initiate an instance

In [21]:
n_titles = titlebank.n_titles
n_words = wordbank.n_words
ninp = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value

model = TransformerModel(n_titles, n_words, ninp, nhead, nhid, nlayers, dropout).to(device)

## Run the Model

### Preparations

In [22]:
import random

In [23]:
def chooseFromDataset(index):
    sample=positionsDataset[index]
    return (sample['input'],sample['target'])

positionsDataset = PositionDataset(train_set)

In [24]:
n_iters = 2000

In [25]:
training_pairs = [chooseFromDataset(random.randint(0,lenTrainSet-1)) for i in range(n_iters)]

In [26]:
evaluating_pairs = [chooseFromDataset(random.randint(0,lenTrainSet-1)) for i in range(n_iters)]

### Define train( ) and evaluate() 

In [27]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [28]:
import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    n_words = wordbank.n_words
    
    for iter_time in range(1, n_iters + 1):
        training_pair = training_pairs[iter_time - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        optimizer.zero_grad()
        output = model(input_tensor)

        loss = criterion(output.view(n_words,-1), target_tensor)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if iter_time % log_interval == 0 and iter_time > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} iters | '
                  'lr {:02.2f} | ms/iter {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, iter_time, n_iters, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

In [29]:
def evaluate(eval_model, evaluating_pairs):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    n_words = wordbank.n_words

    with torch.no_grad():
        for iter_time in range(1, n_iters + 1):
            training_pair = training_pairs[iter_time - 1]
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            output = eval_model(input_tensor)
            output_flat = output.view(n_words,-1)
            total_loss += len(input_tensor) * criterion(output_flat, target_tensor).item()
    return total_loss / len(evaluating_pairs)

### Loop over epochs

In [30]:
best_val_loss = float("inf")
epochs = 3 # The number of epochs
best_model = None

In [31]:
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, evaluating_pairs)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

/opt/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch   1 |   200/ 2000 iters | lr 5.00 | ms/iter 90.55 | loss  1.15 | ppl     3.15
| epoch   1 |   400/ 2000 iters | lr 5.00 | ms/iter 85.98 | loss  1.00 | ppl     2.71
| epoch   1 |   600/ 2000 iters | lr 5.00 | ms/iter 87.06 | loss  1.00 | ppl     2.72
| epoch   1 |   800/ 2000 iters | lr 5.00 | ms/iter 84.96 | loss  1.02 | ppl     2.77
| epoch   1 |  1000/ 2000 iters | lr 5.00 | ms/iter 81.73 | loss  0.95 | ppl     2.58
| epoch   1 |  1200/ 2000 iters | lr 5.00 | ms/iter 80.89 | loss  0.97 | ppl     2.63
| epoch   1 |  1400/ 2000 iters | lr 5.00 | ms/iter 79.86 | loss  0.98 | ppl     2.67
| epoch   1 |  1600/ 2000 iters | lr 5.00 | ms/iter 79.81 | loss  0.94 | ppl     2.57
| epoch   1 |  1800/ 2000 iters | lr 5.00 | ms/iter 80.36 | loss  0.97 | ppl     2.65
| epoch   1 |  2000/ 2000 iters | lr 5.00 | ms/iter 80.15 | loss  0.83 | ppl     2.30
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 173.51s | valid loss  4

In [32]:
testing_pairs = [chooseFromDataset(random.randint(0,lenTrainSet-1)) for i in range(n_iters)]

In [33]:
test_loss = evaluate(best_model, testing_pairs)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

| End of training | test loss  4.61 | test ppl   100.08


In [34]:
test_pair = chooseFromDataset(random.randint(0,lenTrainSet-1))

In [35]:
test_pair

(tensor([[15011],
         [    1]]), tensor([0, 1, 0,  ..., 0, 0, 0]))

In [43]:
input_tensor = test_pair[0]
target_tensor = test_pair[1]
input_tensor.shape

torch.Size([2, 1])

In [69]:
input2 = input_tensor.view(1,-1)

In [70]:
input2

tensor([[15011,     1]])

In [73]:
input3 = torch.tensor([100,200,102]).view(-1,1)
input3

tensor([[100],
        [200],
        [102]])

In [44]:
target_tensor.shape

torch.Size([20187])

In [40]:
output = model(input_tensor)
output

tensor([[[  3.2953,   0.1085,  -1.6837,  ...,   3.4385,   1.5464,   1.6353]],

        [[ -1.7891,  -2.7430,  -3.5916,  ..., -13.7187, -15.8490, -18.4089]]],
       grad_fn=<AddBackward0>)

In [71]:
output2=model(input2)
output2

tensor([[[  3.2953,   0.1085,  -1.6837,  ...,   3.4385,   1.5464,   1.6353],
         [ -1.7877,  -2.7437,  -3.5929,  ..., -13.7170, -15.8496, -18.4093]]],
       grad_fn=<AddBackward0>)

In [41]:
output.size()

torch.Size([2, 1, 20187])

In [72]:
output2.size()

torch.Size([1, 2, 20187])

In [74]:
output3 = model (input3)
output3

tensor([[[ 1.1041, -0.0123, -0.5805,  ...,  0.8220, -0.3597, -0.1910]],

        [[ 0.9940, -0.0144, -0.5152,  ...,  0.7483, -0.3791, -0.1980]],

        [[ 0.9922, -0.0142, -0.5130,  ...,  0.7491, -0.3768, -0.1954]]],
       grad_fn=<AddBackward0>)

In [75]:
output3.size()

torch.Size([3, 1, 20187])

In [94]:
class TransformerModelNew(nn.Module):

    def __init__(self, n_titles, n_words, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModelNew, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout) #get the positional encoder
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout) #get a layer
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers) # get the encoder
        self.encoder = nn.Embedding(n_titles, ninp) #embedding layer
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, n_words)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output_encoder = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output_encoder)
        return src,output_encoder,output

In [95]:
n_titles = titlebank.n_titles
n_words = wordbank.n_words
ninp = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
model1 = TransformerModelNew(n_titles, n_words, ninp, nhead, nhid, nlayers, dropout).to(device)

In [101]:
input_tensor=input_tensor.view(1,-1)
input_tensor.shape

torch.Size([1, 2])

In [106]:
input_tensor

tensor([[15011,     1]])

In [102]:
src,output_encoder,output = model1(input_tensor)

In [98]:
print(f'''
src: {src}
output_encoder: {output_encoder}
output: {output}
''')


src: tensor([[[-5.4347e-02, -3.4511e-01,  1.2058e+00,  2.5655e-01,  0.0000e+00,
           2.8661e+00,  1.4727e-02,  1.5741e+00,  1.3153e+00,  2.4396e+00,
          -1.1892e+00,  8.0882e-01,  0.0000e+00,  0.0000e+00,  8.6663e-01,
           2.4342e+00,  1.6432e+00,  2.8729e+00, -0.0000e+00,  0.0000e+00,
           1.2965e+00, -2.7127e-01, -0.0000e+00,  1.7280e+00,  1.4355e+00,
           1.9397e+00,  1.4039e+00,  0.0000e+00,  0.0000e+00,  1.9300e+00,
          -1.5940e+00,  0.0000e+00,  4.0013e-01,  5.0341e-01,  1.5805e-01,
           0.0000e+00,  1.6739e+00,  0.0000e+00,  0.0000e+00, -3.0044e-01,
           1.3160e+00,  0.0000e+00,  1.6506e+00,  0.0000e+00,  9.5599e-01,
           6.6824e-01, -0.0000e+00,  2.7596e+00, -0.0000e+00, -3.8271e-01,
          -4.7592e-01,  1.0277e+00,  1.1673e+00, -4.4270e-01,  1.1207e+00,
           2.0761e+00, -0.0000e+00,  1.2975e+00, -8.4151e-01,  0.0000e+00,
           1.0235e+00,  0.0000e+00, -4.9217e-01,  1.7472e+00, -2.3958e-01,
           8.7226e-

In [103]:
print(f'''
src: {src.shape}
output_encoder: {output_encoder.shape}
output: {output.shape}
''')


src: torch.Size([1, 2, 200])
output_encoder: torch.Size([1, 2, 200])
output: torch.Size([1, 2, 20187])



In [105]:
output_adjusted = output.view(-1,n_words)
output_adjusted.shape

torch.Size([2, 20187])

## Print out the results

In [107]:
n_print = 10

In [114]:
def evaluateFromDataset(index):
    each_set=train_set[index]
    sample=positionsDataset[index]
    return (each_set['input'],each_set['target'],sample['input'],sample['target'])

In [115]:
printing_pairs=[evaluateFromDataset(random.randint(0,lenTrainSet-1)) for i in range(n_print)]

In [116]:
def evalPrint(eval_model, input_tensor):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    n_words = wordbank.n_words

    with torch.no_grad():
        
        output = eval_model(input_tensor)
        output_flat = output.view(-1,n_words)
        softmax = nn.LogSoftmax(dim=0)
        adjusted = torch.add(output_flat[0],output_flat[1])
        final = softmax(adjusted)
        decoded_words = []
        while True:
            topv, topi = final.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(wordbank.index2word[topi.item()])
                
    return decoded_words

In [120]:
def evaluateRandomly(model, n=10):
    for i in range(n):
        pair = printing_pairs[i]
        print('input sequence: ', pair[0])
        target_title = ' '.join(pair[1])
        print('target title: ', target_title)
        output_words = evalPrint(model, pair[2])
        output_title = ' '.join(output_words)
        print('output title: ', output_title)
        print('-'*80)

In [121]:
evaluateRandomly(best_model)

input sequence:  ['Manager / Consultant', 'Account Manager', 'Senior Manager, Client Services', 'Group Account Manager', 'Unit Head']
target title:  Account Director


KeyboardInterrupt: 